In [ ]:
# Positive Reviews
import os;
with open("./tr_polarity.pos", 'rb') as f:
     # reviews_pos = f.read().decode('iso-8859-9').replace('\r', '');
     reviews_pos = f.read().decode('cp1254').replace('\r', '').splitlines();

reviews_pos

In [ ]:
# Negative Reviews
with open("./tr_polarity.neg", 'rb') as f:
     reviews_neg = f.read().decode('cp1254').replace('\r', '').splitlines();

reviews_neg

In [3]:
# Check Dataset Size Equality
assert(len(reviews_pos) == len(reviews_neg))

In [4]:
# Slice the dataset for testing, cross valitation and training
# 10% for testing
# 90% for training

data_size = len(reviews_pos)
test_size = int(data_size * 0.1)
print("Data size = %d, Test Size = %d, Last Index = %d" % (data_size, test_size, test_size * 10));
test_index = [(x * test_size, x * test_size + test_size) for x in range(10)];
print("Dataset slices for testing: %s" % test_index)

Data size = 5331, Test Size = 533, Last Index = 5330
Dataset slices for testing: [(0, 533), (533, 1066), (1066, 1599), (1599, 2132), (2132, 2665), (2665, 3198), (3198, 3731), (3731, 4264), (4264, 4797), (4797, 5330)]


In [5]:
# Correction: include remaining reviews, if any, to the last index pair
test_index[9] = (test_index[9][0], data_size);
print(test_index[9][0])

4797


In [6]:
# Utility function for dataset splitting
def split_dataset(data, split_indexes):
  data_test  = data[split_indexes[0]:split_indexes[1]];
  data_training = data[0:split_indexes[0]] + data[split_indexes[1]:len(data)];
  return { "test": data_test, "train": data_training};

In [ ]:
# Cross validation
'''
for i in range(10):
    splits = split_dataset(reviews_pos, test_index[i])
    pos_test = splits['test']
    pos_train = splits['train']
    print("Pos test size: %d, \n Last item : %s" % (len(pos_test), pos_test[-1]) )
    print("Pos train size: %d, \n Last item : %s" % (len(pos_train), pos_train[0]) )

    splits = split_dataset(reviews_neg, test_index[i])
    neg_test = splits['test']
    neg_train = splits['train']
    print("Neg test size: %d, \n Last item : %s" % (len(neg_test), neg_test[-1]) )
    print("Neg train size: %d, \n Last item : %s" % (len(neg_train), neg_train[0]) )
'''


In [51]:
import nltk;
import math
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\enest\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [8]:
# Tokenize
reviews_pos_tokens = [nltk.word_tokenize(doc) for doc in reviews_pos];
reviews_neg_tokens = [nltk.word_tokenize(doc) for doc in reviews_neg];

In [9]:
# Validate test split construction
splits = split_dataset(reviews_pos_tokens, test_index[0]);
pos_test = splits['test'];
pos_train = splits['train'];
print("Pos test size: %d, \n Last item : %s" % (len(pos_test), pos_test[-1]) )
print("Pos train size: %d, \n Last item : %s" % (len(pos_train), pos_train[0]) )

splits = split_dataset(reviews_neg_tokens, test_index[0]);
neg_test = splits['test'];
neg_train = splits['train'];
print("Neg test size: %d, \n Last item : %s" % (len(neg_test), neg_test[-1]) )
print("Neg train size: %d, \n Last item : %s" % (len(neg_train), neg_train[0]) )

Pos test size: 533, 
 Last item : ['romantik-komedi', 'severler', 'icin', 'cok', 'iyi', 'bi', 'secim', 'olur', 'cok', 'sicak', 'eglenceli', 'komik', 've', 'iyi', 'oyunculukla', 'birlesince', 'guzel', 'bi', 'film', 'cikmis', 'julia', 'roberts', 'yine', 'oyunculuk', 'dersi', 'vermis', 'tavsiye', 'ederim', '.']
Pos train size: 4798, 
 Last item : ['çok', 'fazla', 'eglenceli', 'bir', 'romantik-komedi', 'idi', '.', 'en', 'fazla', 'gösterimde', 'kalan', ',', 'filmlerden', 'birisi', 'ayrica', '....']
Neg test size: 533, 
 Last item : ['kesinlikle', 'iyi', 'bir', 'psikoloji', 'gerilim', 'bekleyenlerin', 'gerilimlerini', 'karsilamayan', 'bol', 'kliseli', 'vasatin', 'bile', 'altinda', 'bir', 'film', 'filmin', 'tek', 'bir', 'artisi', 'var', 'oda', 'john', 'cusack', 'in', 'müthis', 'oyunculugu', 'bundan', 'öte', '1408', 'de', 'hiç', 'bir', 'sey', 'yok', '.', '.']
Neg train size: 4798, 
 Last item : ['bu', 'filmin', 'neresini', 'begendiniz', 'anlamadim', 'saçma', 'sapan', 'bi', 'gerilim', 'filmi', 

In [ ]:
# Vocabulary
pos_train_flat = [X for x in pos_train for X in x];
V_pos = { x for x in pos_train_flat }; 
print("|V_pos| = %d, \n elements: %s" % (len(V_pos), V_pos))

In [ ]:
neg_train_flat = [X for x in neg_train for X in x];
V_neg = { x for x in neg_train_flat };
print("|V_neg| = %d, \n elements: %s" % (len(V_neg), V_neg))

In [12]:
# 1-gram statistics, i.e. 1-gram LM for POS
N = len(V_pos);
LM_pos = {x:0 for x in V_pos};
for token in pos_train_flat:
  LM_pos[token] += 1;

# Most frequent tokens
mfreq_pos = [(x,LM_pos[x]) for x in LM_pos];
mfreq_pos.sort(key=lambda t: t[1], reverse=1);

print("Most frequent 20 tokens in positive reviews: %s" % (mfreq_pos[:20]))

Most frequent 20 tokens in positive reviews: [('.', 4457), ('bir', 4103), ('film', 3209), ('..', 2350), ('çok', 2011), (',', 1880), ('ve', 1575), ('bu', 1500), ('...', 1188), ('güzel', 1150), ('iyi', 1086), ('filmi', 979), ('en', 935), ('bi', 916), ('ama', 889), ('....', 882), ('!', 693), ('kadar', 635), ('bence', 628), ('daha', 558)]


In [13]:
M = len(V_neg);
LM_neg = {x:0 for x in V_neg};
for token in neg_train_flat:
    LM_neg[token] += 1;

# Most frequent tokens
mfreq_neg = [(x,LM_neg[x]) for x in LM_neg];
mfreq_neg.sort(key=lambda t: t[1], reverse=1);

print("Most frequent 20 tokens in negative reviews: %s" % (mfreq_neg[:20]))

Most frequent 20 tokens in negative reviews: [('.', 4860), ('bir', 3533), ('film', 2715), ('..', 2340), (',', 2028), ('bu', 1884), ('çok', 1542), ('ve', 1268), ('ama', 1116), ('filmi', 1095), ('...', 988), ('kötü', 931), ('kadar', 819), ('....', 788), ('bi', 766), ('!', 724), ('daha', 631), ('ne', 595), ('en', 590), ('hiç', 558)]


In [ ]:
#first 10 elements of the positive
print('Vocabulary: ', list(V_pos)[:20])

In [ ]:
#first 10 elements of the negative
print('Vocabulary: ', list(V_neg)[:20])

In [14]:
vocab_pos = V_pos
vocab_neg = V_neg

In [ ]:
for idx, token in enumerate(vocab_pos):
  print('index = %d \t vocabulary term: %s' % (idx, token))

In [ ]:
for idx, token in enumerate(vocab_neg):
  print('index = %d \t vocabulary term: %s' % (idx, token))

In [ ]:
term2idx_pos = {}
for idx, token in enumerate(vocab_pos):
    term2idx_pos.update({token: idx})

print(term2idx_pos)

In [ ]:
term2idx_neg = {}
for idx, token in enumerate(vocab_neg):
    term2idx_neg.update({token: idx})

print(term2idx_neg)

In [19]:
#Indexes for the terms of individual documents. Documents are the test reviews.

print('Term indexes for review 1 of positive test: %s' % [term2idx_pos.get(token) for token in pos_test[0]])
print('Term indexes for review 1 of negative test: %s' % [term2idx_neg.get(token) for token in neg_test[0]])


Term indexes for review 1 of positive test: [1615, 221, 1947, 7067, 8424, 8906, 2429, 9585, 5414, 12314, 1622, 5366, None, 15828, 3344, 3155, None, 15248, 13481, 3542, 1947, 9799, 5366, 17222, 5318, 13147, 12945]
Term indexes for review 1 of negative test: [10019, 15282, 15371, None, None, 8561, 2058, 13724, None, 17467, 12783, 11476, 8927, 8805, 904, 6159, 17432, 14476]


In [ ]:
# To access terms via index, create a new dictionary from index to terms
idx2term_pos = {}
for term in term2idx_pos:
    idx = term2idx_pos.get(term)
    idx2term_pos.update({idx: term})
print(idx2term_pos)

In [ ]:
idx2term_neg = {}
for term in term2idx_neg:
    idx = term2idx_neg.get(term)
    idx2term_neg.update({idx: term})
print(idx2term_neg)

In [22]:
print(len(vocab_pos))
print(len(vocab_neg))

18306
19830


In [27]:
pos_train_vectors = []
for doc in pos_train:
    doc_vector_pos = [0] * len(vocab_pos)
    for token in doc:
        idx = term2idx_pos.get(token)
        doc_vector_pos[idx] = 1
    pos_train_vectors.append(doc_vector_pos)
print('Number of documents in the positive training set: %d' % len(pos_train_vectors))
print('Number of terms in the positive vocabulary: %d' % len(vocab_pos)) 





Number of documents in the positive training set: 4798
Number of terms in the positive vocabulary: 18306


In [24]:
neg_train_vectors = []
for doc in neg_train:
    doc_vector_neg = [0] * len(vocab_neg)
    for token in doc:
        idx = term2idx_neg.get(token)
        doc_vector_neg[idx] = 1
    neg_train_vectors.append(doc_vector_neg)
print('Number of documents in the negative training set: %d' % len(neg_train_vectors))
print('Number of terms in the negative vocabulary: %d' % len(vocab_neg))




Number of documents in the negative training set: 4798
Number of terms in the negative vocabulary: 19830


In [36]:
pos_test_vectors = []
for doc in pos_test:
    doc_vector_pos = [0] * len(vocab_pos)
    for token in doc:
        if token in vocab_pos:
            idx = term2idx_pos.get(token)
            doc_vector_pos[idx] = 1
    pos_test_vectors.append(doc_vector_pos)

neg_test_vectors = []
for doc in neg_test:
    doc_vector_neg = [0] * len(vocab_neg)
    for token in doc:
        if token in vocab_neg:
            idx = term2idx_neg.get(token)
            doc_vector_neg[idx] = 1
    neg_test_vectors.append(doc_vector_neg)


In [42]:
print(len(pos_test_vectors))
print(len(neg_test_vectors))

533
533


In [43]:
# create a new vector by averaging all vectors in the pos_train_vectors
pos_train_vector_avg = [0] * len(vocab_pos)
for doc_vector in pos_train_vectors:
    for idx, val in enumerate(doc_vector):
        pos_train_vector_avg[idx] += val
for idx, val in enumerate(pos_train_vector_avg):
    pos_train_vector_avg[idx] = val / len(pos_train_vectors)

# create a new vector bu averaging all vectors in the neg_train_vectors
neg_train_vector_avg = [0] * len(vocab_neg)
for doc_vector in neg_train_vectors:
    for idx, val in enumerate(doc_vector):
        neg_train_vector_avg[idx] += val
for idx, val in enumerate(neg_train_vector_avg):
    neg_train_vector_avg[idx] = val / len(neg_train_vectors)



In [45]:
print(len(pos_train_vector_avg))
print(len(neg_train_vector_avg))
print(pos_train_vector_avg[0:10])


18306
19830
[0.00020842017507294707, 0.00020842017507294707, 0.00020842017507294707, 0.00020842017507294707, 0.0010421008753647354, 0.0006252605252188412, 0.012505210504376824, 0.00041684035014589413, 0.00020842017507294707, 0.00020842017507294707]


In [63]:
calculator = math.sqrt(sum([val**2 for val in pos_train_vector_avg]))*math.sqrt(sum([val**2 for val in neg_train_vector_avg]))
resultList = []


for doc_vector in pos_test_vectors:

    numerator_pos = sum([val1*val2 for val1, val2 in zip(doc_vector, pos_train_vector_avg)])
    numerator_neg = sum([val1*val2 for val1, val2 in zip(doc_vector, neg_train_vector_avg)])

    pos_ratio = 0
    neg_ratio = 0
    cos_sim = numerator_pos / calculator
    pos_ratio = cos_sim
    cos_sim = numerator_neg / calculator
    neg_ratio = cos_sim
    if pos_ratio > neg_ratio:
        resultList.append(1)
    else:
        resultList.append(0)

for doc_vector in neg_test_vectors:

    numerator_pos = sum([val1*val2 for val1, val2 in zip(doc_vector, pos_train_vector_avg)])
    numerator_neg = sum([val1*val2 for val1, val2 in zip(doc_vector, neg_train_vector_avg)])

    pos_ratio = 0
    neg_ratio = 0
    cos_sim = numerator_pos / calculator
    pos_ratio = cos_sim
    cos_sim = numerator_neg / calculator
    neg_ratio = cos_sim
    if pos_ratio > neg_ratio:
        resultList.append(1)
    else:
        resultList.append(0)







    







In [61]:
print(resultList)

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [ ]:
#term by document matrix from list of document vectors
matrix = [[doc_vec[j] for doc_vec in pos_train_vectors] for j in range(len(vocab_pos))]

print('\t \t d1 \t d2 \t d3 \t d4 \t d5')
for idx, row in enumerate(matrix):
    print("{}\t  {}\t  {}\t {}\t  {}\t  {}\t  {}".format(idx2term_pos[idx], row[0], row[1], row[2], row[3], row[4], row[5]))

In [ ]:
#Word cloud analysis
from wordcloud import WordCloud
import matplotlib.pyplot as plt

# Word cloud for document 227 and 228
plt.figure(figsize=(16, 16))

doc = ' '.join([w for w in pos_train[227]])
wordcloud = WordCloud().generate(doc)
plt.subplot(1, 2, 1).set_title('Document 227')
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")

doc2 = ' '.join([w for w in pos_train[228]])
wordcloud2 = WordCloud().generate(doc2)
plt.subplot(1, 2, 2).set_title('Document 228')
plt.imshow(wordcloud2, interpolation='bilinear')
plt.axis("off")



plt.show()
